# <font style="color:#0015FF">Intersection Over Union</font>

by:**Amr Abdelhamed**

**[Linkedin profile](https://www.linkedin.com/in/amrabdelhamed69/)**

**part of object-detection series: [Github repo](https://github.com/Amrabdelhamed611/ML_Implementation/tree/main/object_detection)**

Intersection over Union is an evaluation metric for object detectors measure the performance of the object detector. hence,it describe the overlapping of two boxes.

to begin with calculating IOU we need to more about bounding boxes. 


## <font style="color:blue"> Bounding Boxes</font>

Bounding boxes are the coordinates of a rectangle which surrounds an object in an image to describe the location of an object aka (localization).

the Bounding Boxes in object detection represented as coordinates and confidence: 

- prediction bounding box confidence reflects the probability of how likely the box contains an object.
- so the target bounding boxes has confidence of 1 as we sure that object exist in the bounding boxes.

popular Bounding Boxes formats:

- **Pascal VOC dataset format:** where the Bounding Boxes represented by the box corners as `[x_min, y_min, x_max, y_max]` 
- **Yolo format:** where the Bounding Boxes represented by the box center point ,width and height `[x_center, y_center, width, height]`



## <font style="color:blue">Calculating IOU</font>

at the beginning, Let us assume there is 2 boxes represented corners points by blue box [x1, y1, x2, y2] and,red box [x3, y3, x4, y4] respectively.

<img src="./attachments/box1.png" alt="intersection" width="400" height="400"></img>

<div style="page-break-after: always; break-after: page;"></div>

the intersection (box3) is the purple box we can calculate IOU as following:
$$IOU = \dfrac{\text{ Area of intersection of boxes}}{\text {Area of union of boxes}}$$

so, we now have box1 , box2 and intersection box (box3) we need to get the union area and intersection area:

- intersection Area is the purple area of $box3_{Area}$
- union Area is $box1_{Area}+box2_{Area}-box3_{Area}$

the key here is to get box3 coordinates to calculate intersection area then the IOU. let look to different intersection:
<img src="./attachments/box2.png" alt="various intersection scenarios" width="800" height="800"></img>


## <font style="color:blue">Intersection coordinates</font>

form cases above we can calculate intersection box corners points as following:

* $P1_{(x,y)} = (\ max(x_1,x_3),max(y_1,y_3)\ )$
* $P2_{(x,y)} = (\ min(x_2,x_4),min(y_2,y_4)\ )$

form `P1 and P2` we can calculate intersection area , then the IOU. 

<div style="page-break-after: always; break-after: page;"></div>

## <font style="color:blue">IOU Implementation Steps</font>

1. input box1 ,box2 and box format.
2. convert box format to corners format.
3. calculate `P1,P2` as above.
4. calculate intersection area from `P1,P2`.
5. calculate union area (box1_area + box2_area - intersection)
6. calculate the IOU (intersection /union)

**implementation notes:**

- when calculate intersection area set minimum values of intersection to zeros by using clipping functions as the intersection area will be only negative if `P2<P1` mean there is no intersection.
- for all division operation add small  value to the denominator for numerical stability to avoid division by zero. 

In [1]:
import torch 
import torch.nn as nn

def iou(box1,box2,box_format= 'mid',device ='cpu'):
    """
    Calculates intersection over union between to batches of Bounding Boxes. 
    Parameters
    ----------
        box1 : tensor 
            Bounding Boxes (BATCH_SIZE, 4).
        box2 : tensor
            Bounding Boxes (BATCH_SIZE, 4).
        box_format : str
            Bounding Boxes inputed format (default is mid).
            'mid': for Yolo format where x,y is the center of box [x(mid),y(mid),h,w] ,
            'min': format where x,y the top left point of the box [x(min),y(min),h,w] ,
            'corners': for (SSD/ RCNN) format uses coordinates of the top-left corner and bottom-right corner
                        [x(min),y1(min),x2(max),y2(max)]
        device : str
           specify the device type responsible to load a tensor into memory (default is cpu).
    Returns
    -------
        tensor: Intersection over union for all examples
    """  
    # make sure bboxes not empty and in has 4 values 
    assert (isinstance(box1,torch.Tensor) and (box1.shape[-1] ==4) and ((box1.shape[-2]) >0)),\
                    "Box 1 must be tensor with sahpe (BATCH_SIZE,...,4)"
    assert (isinstance(box2,torch.Tensor) and (box2.shape[-1] ==4) and ((box2.shape[-2]) >0)),\
                    "Box 2 must be tensor with sahpe (BATCH_SIZE,...,4)"
    box1= torch.clone(box1).detach().float().to(device)
    box2= torch.clone(box2).detach().float().to(device)
    # bassed on the input format calculate the area of each box to get the union 
    # then convert the box format to corners to get intersction the points 
    if box_format == 'mid':        
        sum_area = torch.mul(box1[...,2:3],box1[...,3:4] ) +  torch.mul(box2[...,2:3],box2[...,3:4] )
        box1[...,0:2],box1[...,2:4]  =(box1[...,0:2]- torch.divide(box1[...,2:4],2) ,
                                        box1[...,0:2]+ torch.divide(box1[...,2:4],2))
        box2[...,0:2],box2[...,2:4]  =(box2[...,0:2]- torch.divide(box2[...,2:4],2) ,
                                        box2[...,0:2]+ torch.divide(box2[...,2:4],2))
        
        
    elif box_format == 'min':
        sum_area = torch.mul(box1[...,2:3],box1[:,3:4] ) +  torch.mul(box2[...,2:3],box2[...,3:4] )+ 1e-8
        box1[...,2:4] =box1[...,0:2]+ box1[...,2:4]
        box2[...,2:4] =box2[...,0:2]+ box2[...,2:4]
        
    elif box_format == 'corners':
        sum_area = torch.mul(box1[...,2:3]-box1[...,0:1] , box1[...,3:4] -box1[...,1:2]) +\
                    torch.mul(box2[...,2:3]-box2[...,0:1] , box2[...,3:4] -box2[...,1:2])+ 1e-8
        
    # boxes intersection points
    p1 =torch.max(box1[...,0:2],box2[...,0:2]).clamp(0)
    p2 =torch.min(box1[...,2:4],box2[...,2:4]).clamp(0)
    intersection=torch.mul( (p2[...,0:1]-p1[...,0:1] ).clamp(0),( p2[...,1:2] -p1[...,1:2]).clamp(0))
    return  intersection/(sum_area-intersection)


In [2]:
# test cases 
'''
test the function to check it works as it should.
i borrowed some test case from Aladdin Persson check out his work in the Resources
'''
tests_b1 = []
tests_b2 = []
results = []
formats =[]
#1------------------------------------------------------
b1= torch.tensor([[2.5,3.5,3,5]]).expand(27,-1).view(3,3,3,4)
b2= torch.tensor([[3.5,6,3,6]]).expand(27,-1).view(3,3,3,4)
r = torch.tensor([0.2222]).expand(27,-1).view(3,3,3,1)
f = 'mid'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#2------------------------------------------------------
b1= torch.tensor([[.25,.35,.3,.5],[.2,.2,.2,.2],[.2,.2,.2,.2]])
b2= torch.tensor([[.35,.6,.3,.6],[.5,.5,.4,.4],[.2,.2,.2,.2]])
r = torch.tensor([[0.2222],[0.0],[1.0]])
f = 'mid'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#3------------------------------------------------------
b1= torch.tensor([0.78, 0.095, 0.2, 0.2]).expand(27,-1).view(3,3,3,4)
b2= torch.tensor([0.88, 0.1, 0.2, 0.2]).expand(27,-1).view(3,3,3,4)
r = torch.tensor([0.3223]).expand(27,-1).view(3,3,3,1)
f = 'mid'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#4------------------------------------------------------
b1=  torch.tensor([0.95, 0.6, 0.5, 0.2]).expand(27,-1).view(3,3,3,4)
b2= torch.tensor([0.95, 0.7, 0.3, 0.2]).expand(27,-1).view(3,3,3,4)
r = torch.tensor([0.2308]).expand(27,-1).view(3,3,3,1)
f = 'mid'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#5------------------------------------------------------
b1= torch.tensor([0.25, 0.15, 0.3, 0.1]).expand(27,-1).view(3,3,3,4)
b2=  torch.tensor([0.25, 0.35, 0.3, 0.1]).expand(27,-1).view(3,3,3,4)
r = torch.tensor([0]).expand(27,-1).view(3,3,3,1)
f = 'mid'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#6------------------------------------------------------
b1=  torch.tensor([0.5, 0.5, 0.2, 0.2]).expand(27,-1).view(3,3,3,4)
b2=  torch.tensor([0.5, 0.5, 0.2, 0.2]).expand(27,-1).view(3,3,3,4)
r = torch.tensor([1]).expand(27,-1).view(3,3,3,1)
f = 'mid'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#7------------------------------------------------------
b1=  torch.tensor([0.7, 0.95, 0.6, 0.1]).expand(27,-1).view(3,3,3,4)
b2=  torch.tensor([0.5, 1.15, 0.4, 0.7]).expand(27,-1).view(3,3,3,4)
r = torch.tensor([0.0968]).expand(27,-1).view(3,3,3,1)
f = 'mid'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#8-----------------------------------------------------
b1= torch.tensor([[2.5,3.5,3,5]]).expand(27,-1).view(3,3,3,4)
b2= torch.tensor([[3.5,6,3,6]]).expand(27,-1).view(3,3,3,4)
r = torch.tensor([0.2222]).expand(27,-1).view(3,3,3,1)
f = 'mid'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#9------------------------------------------------------
b1= torch.tensor([[1,1,3,3],[2,2,5,5],[5,5,5,5]]).repeat(49,1).view(3,7,7,4)          
b2= torch.tensor([[1.2,1.1,3,3],[2,3,2,2],[5,5,5,5]]).repeat(49,1).view(3,7,7,4)       
r = torch.tensor([[0.5319,0.1600,1.0000]]).repeat(49,1).view(3,7,7,1)       
f = 'mid'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#10------------------------------------------------------
b1= torch.tensor([[1,1,3,3],[2,2,5,5],[5,5,5,5]]).repeat(49,1).view(3,7,7,4)
b2= torch.tensor([[2,3,2,2],[1,1,3,3],[0,0,0,0]]).repeat(49,1).view(3,7,7,4)
r = torch.tensor([[0.0612,0.2252,0.0]]).repeat(49,1).view(3,7,7,1)       
f = 'mid'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#11------------------------------------------------------
b1= torch.tensor([[0.3,0.3,0.3,0.3],[3,3,3,3],[3,3,3,3]]).repeat(49,1).view(3,7,7,4)
b2= torch.tensor([[0.3,0.3,0.3,0.3],[2,3,2,2],[0,0,0,0]]).repeat(49,1).view(3,7,7,4)
r = torch.tensor([[1,0.3,0]]).repeat(49,1).view(3,7,7,1)       
f = 'mid'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#12------------------------------------------------------
b1= torch.tensor([[3,3,3,3],[3,3,3,3],[3,3,3,3]]).repeat(49,1).view(3,7,7,4)
b2= torch.tensor([[3,3,3,3],[2,3,2,2],[0,0,0,0]]).repeat(49,1).view(3,7,7,4)
r = torch.tensor([[1,0.3,0]]).repeat(49,1).view(3,7,7,1)       
f = 'mid'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#13------------------------------------------------------
b1= torch.tensor([[.2,.2,.5,.5]]).expand(27,-1).view(3,3,3,4)
b2= torch.tensor([[.2,.2,.6,.5]]).expand(27,-1).view(3,3,3,4)
r = torch.tensor([0.75]).expand(27,-1).view(3,3,3,1)
f = 'corners'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#14------------------------------------------------------
b1=  torch.tensor([[0.2, 0.2, 0.5, 0.5]])
b2=  torch.tensor([[0.2, 0.2, 0.5, 0.5]])
r = torch.tensor([1])
f = 'corners'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#15------------------------------------------------------
b1=  torch.tensor([0.2, 0.2, 0.4, 0.4]).expand(27,-1).view(3,3,3,4)
b2=  torch.tensor([0.4, 0.2, 0.6, 0.4]).expand(27,-1).view(3,3,3,4)
r = torch.tensor([0]).expand(27,-1).view(3,3,3,1)
f = 'corners'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#16-----------------------------------------------------
b1=  torch.tensor([0.2, 0.2, 0.4, 0.4]).expand(27,-1).view(3,3,3,4)
b2=  torch.tensor([0.2, 0.4, 0.4, 0.6]).expand(27,-1).view(3,3,3,4)
r = torch.tensor([0]).expand(27,-1).view(3,3,3,1)
f = 'corners'
tests_b1.append(b1) ,tests_b2.append(b2),results.append(r),formats.append(f)
#------------------------------------------------------

def test_func(tst_idx,box1,box2,trgt,formating = 'mid'): 
    out= iou(box1.clone(),box2.clone(),formating).float().round(decimals= 8)
    trgt = trgt.float().round(decimals= 8)
    assert ( torch.abs(out -trgt) < 1e-4).all() , f"test {tst_idx+1} case faild"
    print(f'test case {tst_idx+1} passed')

    
for idx in range(0,len(tests_b1)):
    test_func(idx,tests_b1[idx],tests_b2[idx], results[idx] ,formating =formats[idx])

test case 1 passed
test case 2 passed
test case 3 passed
test case 4 passed
test case 5 passed
test case 6 passed
test case 7 passed
test case 8 passed
test case 9 passed
test case 10 passed
test case 11 passed
test case 12 passed
test case 13 passed
test case 14 passed
test case 15 passed
test case 16 passed


**Resources:**
* [Intersection Over Union by Andrew Ng (DeepLearning.AI)](https://www.youtube.com/watch?v=ANIzQ5G-XPE)
* [Intersection over Union Explained and PyTorch Implementation by Aladdin Persson](https://www.youtube.com/watch?v=XXYG5ZWtjj0&list=PLhhyoLH6Ijfw0TpCTVTNk42NN08H6UvNq&index=2)
* [IOU (Intersection over Union) by Vineeth S Subramanyam ](https://medium.com/analytics-vidhya/iou-intersection-over-union-705a39e7acef?utm_source=pocket_mylist)
* [Intersection over Union (IoU) for object detection by Adrian Rosebrock ](https://pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/)